Analysis of pupil data from the confidenced project
------------------------------------------------

Goal: Design a linear model that explains pupil data from the confidence project. 

Strategy: Use one example subject to design a model and then compare model predictions to event related averages

In [1]:
import cPickle
from conf_analysis import pupil, patsy_transforms as pt
from pylab import *
import seaborn as sns
import pandas as pd
%load_ext autoreload
%autoreload 2
%matplotlib inline
sns.set_style('ticks')

/Users/nwilming/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Load data
--------


In [2]:
# Reload the data
%time events = pd.read_hdf('../temp_data/allevents.h5', 'events')
# Change subject encoding from string to int
index = events.index.names
events = events.reset_index()
events.subject = [int(s[1:]) for s in events.subject.values]
events = events.set_index(index)

CPU times: user 34.3 s, sys: 21.3 s, total: 55.6 s
Wall time: 1min 5s


In [3]:
%time messages = pd.read_hdf('../temp_data/allmessages.h5', 'messages')
index = messages.index.names
messages = messages.reset_index()
messages.subject = [int(s[1:]) for s in messages.subject.values]
messages = messages.set_index(index)

CPU times: user 248 ms, sys: 790 ms, total: 1.04 s
Wall time: 1.27 s


In [4]:
'''
idx = pd.IndexSlice
events = events.loc[idx[2:3, :, 'S04'],:]
messages = messages.loc[idx[2:3, :, 'S04'],:]
'''

"\nidx = pd.IndexSlice\nevents = events.loc[idx[2:3, :, 'S04'],:]\nmessages = messages.loc[idx[2:3, :, 'S04'],:]\n"

In [5]:
low, pafilt, above = pupil.filter_pupil(events.pac, 100, highcut = 10., lowcut = 0.01)

In [6]:
events['pafilt']=pafilt
%time number, time = pupil.fasta2(events, messages, field='decision_onset_time', pre=2, post=3)
events['dts_num'] = number
events['dts_time'] = time
#res = pupil.alignto(events, messages, 'decision_start_time', Hz=100, pre=-1., post=3., event_val='decision')

KeyError: 'the label [decision_onset_time] is not in the [columns]'

In [ ]:
number, time = pupil.fasta2(events, messages, field='trialid_time', pre=0.5, post=0)
events['base_num'] = number
events['base_time'] = time
#baseline = pupil.alignto(events, messages, 'trialid_time', Hz=100, pre=0.5, post=0, event_val='decision')

In [ ]:
# Align trial numbers with those from message struct.
trialvs = pupil.expand_field(events, messages, 
                             messages.index.get_level_values('trial'),  
                             'trialid_time', 'TRIALEND_time')

In [ ]:
events['trial'] = trialvs

In [ ]:
idx = pd.IndexSlice

In [ ]:
mc = [mean(m) for m in messages.contrast]
messages['mc'] = mc

In [ ]:
# Align with trial information
def redux(ev):
    '''
    Reduce a series to either maximum or single unique value
    '''
    u = unique(ev)
    if len(u) == 1:
        return u[0]
    else:
        if ev.name == 'trial':
            return ev.mode()
        return nanmax(u)


    
a = events.drop([u'blink', u'gavx', u'gavy', u'left_gx', u'left_gy', u'pa', u'right_gx',
       u'right_gy', u'start', u'time', u'type',
       u'pafilt', u'palow', u'pahigh', u'sfname',
       u'pac', u'dts_time', u'base_time'], axis=1)

# Reduce colums to max or single unique value
%time d = a.reset_index().groupby('dts_num').agg(redux) 

d = d.merge(
            messages.drop(['decision', 'contrast', 'SYNCTIME', u'SYNCTIME_start', u'TRIALEND', u'TRIALEND_time', u'contrast', u'contrast_on', u'contrast_time', u'decision', u'decision_start', u'decision_start_time', u'decision_time',
            u'feedback', u'feedback_time', u'trialid ', u'trialid_time' ], axis=1).reset_index(), 
            on=['subject', 'block', 'session', 'trial']
           )
# Merge pivot table with trial information
evt_t = events.pivot_table(values='pafilt', columns='dts_num', index='dts_time')
evt_t = evt_t.T.merge(d, left_index=True, right_index=True).set_index(keys=list(d.columns), append=True)
base_t = events.pivot_table(values='pafilt', columns='base_num', index='base_time')
base_t = base_t.T.merge(d, left_index=True, right_index=True).set_index(keys=list(d.columns), append=False)

In [ ]:
figure(figsize=(20, 2.5))
for cnt, (cond, data) in enumerate(evt_t.groupby(level=['decision'])):
    if cond==0 or cond==88:
        continue
    subplot(1,6,cnt+1)
    h,w = data.values.shape
    time = base_t.T.index.values    
    imshow(data.values, aspect='auto', interpolation='none', vmin=-2, vmax=2, extent=[time[0], time[-1], 0, h])
    yticks([0, h])
sns.despine()
tight_layout()

In [ ]:
#Test that filtering on decision outcome in base_t and evt_t gives same trials
assert all(base_t.query('decision>0 & decision < 88').index.get_level_values('base_num') == 
           evt_t.query('decision>0 & decision < 88').index.get_level_values('base_num'))

In [ ]:
def pdiff(et, bt):
    base = bt.values
    base = nanmean(base, 1)
    evt = et.values
    cdiff = evt-base[:, newaxis]
    return cdiff

In [ ]:
figure(figsize=(20, 5))
gb = ['decision', 'feedback']
colors = ['r', 'b']
for i, ((e, et), (b, bt)) in enumerate(zip(evt_t.query('decision>0 & decision < 88').groupby(level=gb), 
                                           base_t.query('decision>0 & decision < 88').groupby(level=gb))):
    subplot(1,4, (i+2)/2)
    ses = array(
            [nanmean(pdiff(sset, sbt), 0) 
                 for (se, sset), (sb, sbt) in zip(et.groupby(level='subject'), 
                                                  bt.groupby(level='subject'))])
    
    #plot(et.T.index, ses.T, color=colors[int(e[1])], alpha=0.1)
    m = ses.mean(0)
    sem = ses.std(0)/(ses.shape[0]**.5)
    fill_between(et.T.index.values.astype(float), y1=m-sem, y2=m+sem, color=colors[int(e[1])], alpha=0.5)
    plot(et.T.index, m, color=colors[int(e[1])], alpha=1, label=e)

    legend()
    ylim([-0.25, 1.1])
    xlim([-2.2, 3.2])
    #axvline(0, color='k')
    #yticks([])
    #plot([-2.2, -2.2], [-0.05, 0.05], lw=5, color='k')
tight_layout()
sns.despine(left=False, trim=True)


In [ ]:
contrast_perc = prctile(evt_t.index.get_level_values('mc'), [0, 20, 40, 60, 80, 100])

figure(figsize=(20, 5))
gb = ['decision', 'feedback']
colors = ['r', 'b']
for i, (low, high) in enumerate(zip(contrast_perc[:-1], contrast_perc[1:])):
    
    et = evt_t.query('%f<mc & mc<=%f'%(low, high))
    bt = base_t.query('%f<mc & mc<=%f'%(low, high))
    subplot(1,5, i+1)
    ses = array(
            [nanmean(pdiff(sset, sbt), 0) 
                 for (se, sset), (sb, sbt) in zip(et.groupby(level='subject'), 
                                                  bt.groupby(level='subject'))])
    
    #plot(et.T.index, ses.T, color=colors[int(e[1])], alpha=0.1)
    m = ses.mean(0)
    sem = ses.std(0)/(ses.shape[0]**.5)
    fill_between(et.T.index.values.astype(float), y1=m-sem, y2=m+sem, color=colors[int(e[1])], alpha=0.5)
    plot(et.T.index, m, color=colors[int(e[1])], alpha=1)

    legend()
    ylim([-0.25, 1.1])
    xlim([-2.2, 3.2])
    #axvline(0, color='k')
    #yticks([])
    #plot([-2.2, -2.2], [-0.05, 0.05], lw=5, color='k')
tight_layout()
sns.despine(left=False, trim=True)

The Model
---------

I use a simple GLM and convolve regressors with JW's standard IRF. I first compute the convolution kernels and then add some event fields that are useful for the model.

In [ ]:
import pupil
yI, ydtI, ydnI = pupil.IRF_pupil()
IRFS = [yI/yI.std(), ydnI/ydnI.std(), ydtI/ydtI.std()]
yIe, ydtIe, ydnIe = pupil.IRF_pupil(tmax=0.25, n=1)#tmax=.25, n=8)
IRFSe = [yIe, ydtIe, ydnIe]

In [ ]:
low, pa , above = pupil.filter_pupil(events.pa, 100, highcut=10, lowcut=0.1)
figure(figsize=(20, 6))
plot(pa)
low, pa, above = pupil.filter_pupil(events.pa, 100, highcut=10, lowcut=1/(6.))
plot(pa)
xlim([50000, 70000])

In [ ]:
events['pafilt'] = pa

# Full GLM

In [ ]:
events['decend'] = events.decision>0

m, yh, y, X, res = pupil.eval_model('''
pafilt ~   
     pt.Z(left_gx) +pt.Z(left_gy) +
     pt.MF(pt.Z(left_gx), IRFS) + 
     pt.MF(pt.Z(left_gy), IRFS) + 
     pt.MF(blink, IRFS) +
     pt.MF(pt.Z(pt.dt(left_gx)), IRFS) + 
     pt.MF(pt.Z(pt.dt(left_gy)), IRFS) +
     
     pt.MF(ref, func=IRFS) +  
     pt.MF(contrast, func=IRFS) +
     pt.MF(pt.Z(pt.dt(contrast)), func=IRFS) +
     
     pt.MF(decramp21, IRFS) +
     pt.MF(decramp22, IRFS) +
     pt.MF(decramp23, IRFS) +
     pt.MF(decramp24, IRFS) +

     pt.MF(dec_start, IRFS) +
     pt.MF(decend, IRFS) +     
     
     pt.MF(feedback_offset_pos, IRFS) +
     pt.MF(feedback_offset_neg, IRFS) 
''', events.reset_index())
print events.shape
print yh.shape
events['yhat'] = yh
events['residuals'] = y-yh


In [ ]:
import glm_viz
figure(figsize=(20, 6))
glm_viz.timecourse(events, y, yh, [2000, 2100])

In [ ]:
messages['contrast_onset_time'] = [x.contrast_time[0] for _,x in messages.iterrows()]
messages['reference_onset_time'] = [x.decision_time-1900 for _, x in messages.iterrows()]

In [ ]:
figure(figsize=(20,8))
glm_viz.condition_averages(events, messages)

In [ ]:
t = linspace(0, 1, 100)
plot(1/t, t)

In [ ]:
xvals = events.pafilt-events.yhat
print xvals.shape

In [ ]:
xcorr = correlate(xvals[::2], xvals[::2], 'same')

In [ ]:
print xcorr.shape
plot((arange(len(xcorr))-len(xcorr)/2.)/50., xcorr)
xlim([-200, 200])